<a href="https://colab.research.google.com/github/jeawoo0594/colabproject/blob/main/tabular_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.axes_grid1 import make_axes_locatable
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

In [ ]:
train = pd.read_csv('/content/drive/My Drive//tabular/train.csv')
test = pd.read_csv('/content/drive/My Drive//tabular/test.csv')
train

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
train.describe()

In [ ]:
train['cat9'].value_counts()

In [ ]:
test['cat9'].value_counts()

In [ ]:
for category,proportion in train['cat9'].value_counts(normalize=True).iteritems():
    print(f'cate: {category}, pro: {proportion}')

In [ ]:
for category,proportion in test['cat9'].value_counts(normalize=True).iteritems():
    print(f'cate: {category}, pro: {proportion}')

In [ ]:
train.isnull().sum()

In [ ]:
train.corr(method='pearson')

In [ ]:
#heatmap으로 상관관계를 표시
import seaborn as sb
plt.rcParams["figure.figsize"] = (40,40)
sb.heatmap(train.corr(),
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = -1, vmax=1 , #컬러차트 영역 -1 ~ +1
          )

In [ ]:
#-----dnn---------
cat_cols = train.columns[1:11]
cat_cols

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, StandardScaler,RobustScaler
class Preprocessor():
    def __init__(self):
        self.en_dic = None
        self.standard_scaler = None
        self.num_cols = None
        self.cat_cols = None
        # self.test_cats_onehot,self.test_onehot_cols = self.cats_onehot(test_cats_en)
    def preprocess(self,data_df,train=True,combine_min_cats=False, add_pca_feats=False):

        if train:
            self.train_ids = data_df.loc[:, 'id']
            train_cats = data_df.loc[:, data_df.dtypes == object]
            self.cat_cols = train_cats.columns

            if combine_min_cats:
                self._find_minority_cats(train_cats)
                train_cats = self._combine_minority_feats(train_cats)

            self.en_dic = defaultdict(LabelEncoder)
            train_cats_en = train_cats.apply(lambda x: self.en_dic[x.name].fit_transform(x))
            tr_cats_onehot,tr_onehot_cols = self.cats_onehot(train_cats_en)
            
            train_num = data_df.loc[:,data_df.dtypes != object].drop(columns=['target','id'])
            self.num_cols = train_num.columns
            self.Standard_Scaler = StandardScaler()
            train_num_scaler = self.Standard_Scaler.fit_transform(train_num)

            if add_pca_feats:
                pca_feats = self._return_num_pca(train_num_scaler)
                X = pd.DataFrame(np.hstack((tr_cats_onehot,pca_feats)),columns=list(tr_onehot_cols)+list(self.pca_cols))
            else:
                X = pd.DataFrame(np.hstack((tr_cats_onehot,train_num_scaler)),columns = list(tr_onehot_cols)+list(self.num_cols))

        else:
            self.test_ids = data_df.loc[:,'id']
            test_cats = data_df.loc[:, cat_cols]
            if combine_min_cats:
                self._find_minority_cats(test_cats)
                test_cats = self._combine_minority_feats(test_cats)
                
            test_cats_en = test_cats.apply(lambda x: self.en_dic[x.name].fit_transform(x))
            test_cats_onehot,test_onehot_cols = self.cats_onehot(test_cats_en)
            test_num = data_df.loc[:,self.num_cols]
            test_num_scaler = self.Standard_Scaler.fit_transform(test_num)

            if add_pca_feats:
                pca_feats = self._return_num_pca(test_num_scaler,train=False)

                X = pd.DataFrame(np.hstack((test_cats_onehot,pca_feats)),columns = list(test_onehot_cols)+list(self.pca_cols))
            
            else:
                X = pd.DataFrame(np.hstack((test_cats_onehot,test_num_scaler)),columns = list(test_onehot_cols)+list(self.num_cols))

        return X

    def cats_onehot(self,data_df):
        self.cats_df = pd.get_dummies(data=data_df,columns=self.cat_cols, prefix= self.cat_cols)
        self.cats_onehot_cols = self.cats_df.columns
        return self.cats_df, self.cats_onehot_cols
   
    def _find_minority_cats(self, data_df):
        self.composite_category = 'z'
        self.threshold = 0.05
        self.minority_col_dict = {}
        self.minority_map_dic = {}
        for feature in cat_cols:
            self.minority_col_dict[feature] = []
            self.minority_map_dic[feature] = {}
            
            for category,proportion in data_df[feature].value_counts(normalize=True).iteritems():
                if proportion < self.threshold:
                    self.minority_col_dict[feature].append(category)
                    self.minority_map_dic[feature] = { x : self.composite_category for x in self.minority_col_dict[feature]}
        return self.minority_map_dic, self.minority_col_dict
    
    def _combine_minority_feats(self, data_df, replace = False):
        new_df = data_df.copy()
        for feat in self.cat_cols:
            col_label = f"{feat}" if replace else f"{feat}_new"
            new_df[feat] = new_df[feat].replace(self.minority_map_dic[feat])
        return new_df

    def _return_num_pca(self,num_df,train=True):
        self.n_components = 0.85
        if train:
            self.pca = PCA(n_components = self.n_components)
            
            num_rd = self.pca.fit_transform(num_df)
            print(f'pca_explain: {self.pca.explained_variance_ratio_}')
            self.pca_cols = [f'pca_{x}' for x in range(num_rd.shape[1])]

        else:
            num_rd = self.pca.transform(num_df)

            self.pca_cols = [f'pca_{x}' for x in range(num_rd.shape[1])]
        
        return pd.DataFrame(num_rd, columns = self.pca_cols)






In [ ]:
data_proc = Preprocessor()
X = data_proc.preprocess(train, combine_min_cats=False, add_pca_feats=True)
y = train.loc[:, 'target']
X_test = data_proc.preprocess(test,train=False,combine_min_cats=False,add_pca_feats=True)
pd.set_option('display.max_columns', 500)
print(X)

In [ ]:
X_test['cat6_7'] = 0
X_test

In [ ]:
X_test.columns

In [ ]:
X_test = X_test[['cat0_0', 'cat0_1', 'cat1_0', 'cat1_1', 'cat2_0', 'cat2_1', 'cat3_0',
       'cat3_1', 'cat3_2', 'cat3_3', 'cat4_0', 'cat4_1', 'cat4_2', 'cat4_3',
       'cat5_0', 'cat5_1', 'cat5_2', 'cat5_3', 'cat6_0', 'cat6_1', 'cat6_2',
       'cat6_3', 'cat6_4', 'cat6_5', 'cat6_6', 'cat6_7', 'cat7_0', 'cat7_1', 'cat7_2',
       'cat7_3', 'cat7_4', 'cat7_5', 'cat7_6', 'cat7_7', 'cat8_0', 'cat8_1',
       'cat8_2', 'cat8_3', 'cat8_4', 'cat8_5', 'cat8_6', 'cat9_0', 'cat9_1',
       'cat9_2', 'cat9_3', 'cat9_4', 'cat9_5', 'cat9_6', 'cat9_7', 'cat9_8',
       'cat9_9', 'cat9_10', 'cat9_11', 'cat9_12', 'cat9_13', 'cat9_14',
       'pca_0', 'pca_1', 'pca_2', 'pca_3', 'pca_4', 'pca_5', 'pca_6', 'pca_7',
       'pca_8']]
X_test

In [ ]:
# features_drop= ['cat6_7']
X_test = X_test.drop(X_test['cat6_7'],axis=1)
X = X.drop(X['cat6_7'],axis=1)
X

In [ ]:
X_test

In [ ]:
X = X.drop('cat6_7',axis=1)
X

In [ ]:
print(f'xshape: {X.shape} \n yshape: {y.shape} \n X_test: {X_test.shape}')

In [ ]:
for tr,te in zip(X.columns,X_test.columns):
    if tr != te:
        print(f'tr: {tr} \n te: {te}')
    else:
        print(("success"))

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=12)
# X_train.shape

In [ ]:
import keras
from keras.layers import Dense, Embedding, Flatten, LSTM, GRU, \
        SpatialDropout1D, Bidirectional, Conv1D, MaxPooling1D, BatchNormalization
from keras.models import Sequential, load_model
from keras import models
from keras import layers
from sklearn.model_selection import KFold
epoch = 100
def aggregate_lr_rate(epoch, lr):

    if epoch < 20:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def dnn_model(lr=aggregate_lr_rate(epoch,0.001), input_feat_dim=X.shape[1]):
    """ Create a Deep NN for regression that supports monte carlo dropout
        and uses Batch Norm """
    model = Sequential()
    
    model.add(layers.Dense(256, activation='relu', input_shape=(input_feat_dim,), 
                           kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(layers.Dropout(0.2))
        
    # regression output layer - no activation
    model.add(layers.Dense(1))
        
    model.compile(optimizer=keras.optimizers.adam(lr=lr), 
                  loss='mse', metrics=['mse'])
    
    return model

In [ ]:
tr_data = [X,y]
tr_data

In [ ]:
import numpy as np

k=5
num_val_samples = len(X)//k
num_epoch = 100
all_scores = []
np.random.shuffle(tr_data)
for i in range(k):

    print(f'처리중인 폴드 : {i+1}')
    # np.random.shuffle(tr_data)

    val_data = X[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = y[i*num_val_samples: (i+1)*num_val_samples]

    partition_tr = np.concatenate([X[:i*num_val_samples],
                              X[(i+1)*num_val_samples: ]], axis=0)
    partition_tr_target = np.concatenate([y[:i*num_val_samples],
                              y[(i+1)*num_val_samples: ]], axis=0)
    model_1 = dnn_model()
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=20)
    history = model_1.fit(partition_tr, partition_tr_target, epochs=num_epoch, batch_size=256, 
                      callbacks=[early_stop],validation_data=(val_data, val_targets))
    val_mse = history.history['val_mse']   #evaluate 함수는 검증결과를 출력할수있다
    all_scores.append(val_mse)

In [ ]:
# np.mean(all_scores)
print(all_scores)

In [ ]:
test_pred = model_1.predict(X_test)
test_pred

In [ ]:
# save submission in csv format
import datetime
timetoday = datetime.datetime.today()
today = [timetoday.day,timetoday.hour,timetoday.minute]
submission_df = pd.read_csv('/content/drive/My Drive//tabular/sample_submission.csv')
submission_df['target'] = test_pred
submission_df.to_csv(f'submission_tabular_{today}.csv',index=False)
!ls
from google.colab import files
files.download(f'submission_tabular_{today}.csv')

In [ ]:
def plot_results(history):
    metric = 'mse'
    figsize = (12,5)

    tr_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(tr_loss))

    fig = plt.figure(figsize=figsize)

    ax = fig.add_subplot(1,1,1)
    plt.plot(epochs,tr_loss,marker='o',label = 'train')
    plt.plot(epochs,val_loss,marker='x',label = 'val')

    plt.title('train/val')
    ax.set_ylabel('loss')
    ax.set_xlabel('epoch')
    plt.legend(loc='best')
    plt.show()
plot_results(history)

In [ ]:
print(history.history('loss'))

In [ ]:
def pred_mc(model, test_inputs):
    
    # pred_probs = [model.predict(test_inputs) for samples in range(n_samples)]
    pred_probs = model.predict(test_inputs)
    return np.mean(pred_probs, axis=0)

test_preds_1 = pred_mc(model_1, X_test)

In [ ]:
# save submission in csv format
submission_df = pd.read_csv('/content/drive/My Drive//tabular/sample_submission.csv')
submission_df['target'] = test_preds_1
submission_df.to_csv('submission_tabular.csv',index=False)
!ls
from google.colab import files
files.download('submission_tabular.csv')